# <center> La course à la rapidité<br/> Projet de la CesiCDP </center>

![logocesi](https://lyon.cesi.fr/wp-content/uploads/sites/30/2020/04/Sans-titre-2-02-500x224.png)

<div style="text-align:right">
    <strong> Développeurs </strong> : Jacob Antonin ; Keurinck Théo ; <br/> Larose Hugo ; Devrouete Corentin ; Genty Aurélien
</div>

<br/><br/>



## <center>I - Introduction </center>

<br/>
En cette ère du XXIème siècle, nous vivons dans une société ou le rendement est mis à rude épreuve. Dans notre société actuelle, être <strong> le plus rapide est devenu une contrainte auquel tout le monde est confronté </strong>.

De plus depuis les années 90, il y a eu une véritable prise de conscience mondiale de la nécessité de réduire la consommation d’énergie et des émissions de gaz à effet de serre. Les premiers engagements sont apparus lors de la signature du protocole de Kyoto en 1997. Mais son entrée en vigueur n’a finalement eu lieu qu’en 2005 et de nombreux scientifiques ont jugé les efforts insuffisants pour ralentir le réchauffement climatique. Depuis, d’autres engagements plus ambitieux ont vu le jour (division par 4 des émissions d’ici 2050 pour la France par exemple, engagements de certaines grandes villes comme Paris). Mais la tâche est compliquée. Les pouvoirs publics et les collectivités territoriales n’ont pas la possibilité d’obliger les entreprises et les particuliers à changer leurs habitudes pour atteindre ces objectifs. L’action se porte donc avant tout à faire évoluer les comportements. L’économie et le recyclage des matières premières, l’amélioration des modes de transports et des performances énergétiques des bâtiments doivent devenir des priorités.

Dans ce sens, <strong> L’ADEME </strong> (Agence de l’Environnement et de la Maîtrise de l’Energie) a récemment lancé un appel à manifestation d’intérêt pour promouvoir la réalisation de démonstrateurs et d’expérimentations de nouvelles solutions de mobilité pour les personnes et les marchandises adaptées à différents types de territoires. 

Notre structure <strong> CesiCDP </strong> est déjà bien implantée dans le domaine. Aidé de nombreux partenaires, Nous avons réalisé plusieurs études sur le thème de la Mobilité Multimodale Intelligente. Les nouvelles technologies de transport, plus économiques et moins polluantes ne sont pas sans poser de nouveaux défis notamment d’un point de vue de l’optimisation de la gestion des ressources. Mais ces problèmes de logistique du transport présentent un enjeu majeur pour l’avenir : ses applications sont nombreuses (distribution du courrier, livraison de produits, traitement du réseau routier, ramassage des ordures) et leur impact sur l’environnement peut être véritablement significatif. 

<strong> <em> Notre étude s’inscrit donc dans le cadre d’une réponse à l’appel de l’ADEME. </em> </strong>

Notre projet sert donc à optimiser une tournée de livraison, de facon à obtenir une durée de transfert minimale entre son point de départ et d'arrivée, tout en gérant le traffic sur 



 ## <center>II - Définition des variables de décision et de la fonction objectif </center>
 
 

### <center>1 - Variables de décision </center>

Tout d'abord, lors de ce projet il nous a fallu définir les variables de décision.

### <center>2 - Fonction objectif </center>

In [ ]:
import random
import os
import string 
import json

#init de la liste contenant les villes
city = {}

# generation des villes une par une 
def generate_city(nb_ville_generer):

    for i in range (0,nb_ville_generer):

         city_name = generate_name(8)
         city_position = generate_position(nb_ville_generer)
         index = 'ville ' + str(i +1 ) 

         city[index] = {'nom_ville': city_name,  'position' : city_position}
         

    return city

#generation d'un nom de ville 
def generate_name(stringLenght):
    lettres = string.ascii_lowercase
    name = ''.join(random.choice(lettres) for i in range(stringLenght))
    return name 

#generation des position de la ville 
def generate_position(nb_ville_generer):
    start_rand = 0
    stop_rand = nb_ville_generer*4

    x = random.randrange(start_rand, stop_rand)
    y = random.randrange(start_rand, stop_rand)

    position = [x,y]
    
    return position



def createFile(filename, nb_ville_generer):   
    dump = generate_city(nb_ville_generer)
    filename_json = 'json_generate/' + filename + '.json' 
    # path_create = '/Livrable - 2/json_generate/'+ filename_json
    # with open(path_create, 'w', encoding='utf-8') as f: 
    #     json.dump(dump, f, ensure_ascii=False)

    f = open(filename_json, "w+")
    json.dump(dump, f, ensure_ascii=False)

    f.close()



print("---/ Bienvenue dans le générateur de ville \--- \n")
nb_ville_generer = int(input("Veuillez entrer le nombre de ville que vous voulez créer ... \n"))
filename = str(input("Veuillez entrer le nom de fichier que vous voulez créer ... \n"))
print("creation du fichier en cours . . . \n")
createFile(filename, nb_ville_generer)
print("Le fichier à bien été créé, à bientot...")

---/ Bienvenue dans le générateur de ville \--- 



### <center>3 - Implémentation de la matrice d'adjacence </center>

In [8]:
import json
import os
import math
from pprint import pprint

#init des villes
ville = None

nom_fichier = input("Veuillez mettre le nom du fichier json a importer\n") + ".json"

#import du JSON de la ville
path_fichier = os.path.join("json_generate",nom_fichier)
with open(path_fichier, "r") as f : 
    ville = json.load(f)


#fonction d'implémentation de la matrice d'adjacence
def create_matrice(ville):

    matrice_adjacence = []

    for ville_a in ville.items():
        element_matrice = []
        
        x_a =  ville_a[1]['position'][0]
        y_a = ville_a[1]['position'][1]
        nom_ville_a = str(ville_a[1]['nom_ville'])

        for ville_b in ville.items():
           x_b = ville_b[1]['position'][0]
           y_b = ville_b[1]['position'][1]
           nom_ville_b = str(ville_b[1]['nom_ville'])

           if nom_ville_a == nom_ville_b:
               element_matrice.append(0)

           else:
               element_matrice.append(math.sqrt(math.pow(x_b - x_a, 2) + math.pow(y_b - y_a, 2)))

        matrice_adjacence.append(element_matrice)
    return matrice_adjacence


pprint(create_matrice(ville))


Veuillez mettre le nom du fichier json a importer
test10
[[0,
  20.12461179749811,
  16.0312195418814,
  3.605551275463989,
  20.223748416156685,
  22.02271554554524,
  25.059928172283335,
  17.46424919657298,
  13.892443989449804,
  24.041630560342615],
 [20.12461179749811,
  0,
  10.198039027185569,
  23.706539182259394,
  38.47076812334269,
  23.021728866442675,
  31.575306807693888,
  27.313000567495326,
  30.066592756745816,
  43.60045871318328],
 [16.0312195418814,
  10.198039027185569,
  0,
  19.026297590440446,
  36.22154055254967,
  13.038404810405298,
  21.37755832643195,
  17.4928556845359,
  29.120439557122072,
  36.6742416417845],
 [3.605551275463989,
  23.706539182259394,
  19.026297590440446,
  0,
  17.72004514666935,
  23.323807579381203,
  25.0,
  17.204650534085253,
  12.727922061357855,
  20.518284528683193],
 [20.223748416156685,
  38.47076812334269,
  36.22154055254967,
  17.72004514666935,
  0,
  40.718546143004666,
  40.607881008493905,
  33.015148038438355,
  8.

### <center>4 - Représentation d'un graphe via une matrice d'adjacence </center>

## <center>III - Modélisation du problème <center>

### <center>1 - Complexité théorique <center>

Le problème est NP-Complet !


## <center>IV - Modélisation algorithmique <center>

## <center>V - Etude statistique <center>